In [9]:
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [10]:
dataset = get_dataset('solar_nips', regenerate=False)
train_grouper = MultivariateGrouper(max_target_dim=min(2000, int(dataset.metadata.feat_static_cat[0].cardinality)))
data = train_grouper(dataset.test)

In [11]:
data = data[0]['target'].transpose(1,0)
data.shape

(7177, 137)

In [12]:
train = data[:6985, :]
test = data[6985:,:]
train.shape, test.shape

((6985, 137), (192, 137))

In [13]:
scaler = StandardScaler()

train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

train_scaled.shape, test_scaled.shape

((6985, 137), (192, 137))

In [14]:
train_scaled_ec = np.zeros((6985,140))
train_scaled_ec[:,0:137] = train_scaled
train_scaled_ec[:,137:] = train_scaled[:,0:3]

test_scaled_ec = np.zeros((192,140))
test_scaled_ec[:,0:137] = test_scaled
test_scaled_ec[:,137:] = test_scaled[:,0:3]

train_scaled_ec.shape, test_scaled_ec.shape

((6985, 140), (192, 140))

In [15]:
train_scaled_ec_split = np.array(np.split(train_scaled_ec[0:6912], 36, 0))
train_scaled_ec_split.shape

(36, 192, 140)

In [18]:
train_split = np.array(np.split(train_scaled_ec_split, 2, 2)) # use for training 2 batches of size 36
train_split.shape

(2, 36, 192, 70)

In [19]:
test_split = np.array(np.split(np.expand_dims(test_scaled_ec,0), 2, 2)) # user for testing 2 batches of size 1
test_split.shape

(2, 1, 192, 70)

In [ ]:
path_to_real1 = ''
path_to_real2 = ''

path_to_imputation1 = ''
path_to_imputation2 = ''

In [ ]:
real1 = np.load(path_to_real1)[0].transpose(1,0)
real2 = np.load(path_to_real2)[0].transpose(1,0)

impu1 = np.load(path_to_imputation1)[0].transpose(1,0)
impu2 = np.load(path_to_imputation2)[0].transpose(1,0)

In [ ]:
real = np.concatenate([real1, real2], 1)
impu = np.concatenate([impu1, impu2], 1)

realun = scaler.inverse_transform(real[:,0:137]) # use only the first 137 (3 were added)
impuun = scaler.inverse_transform(impu[:,0:137])

In [ ]:
mean_squared_error(realun[168:,:], impuun[168:,:]) # evaluate only on the forecasted area 